<a href="https://colab.research.google.com/github/deepskandpal/hands-on-machine-learning/blob/POC/Newyork_taxi_ETA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [1]:
from google.colab import files
files.upload()


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"atdeepanshu","key":"efb7cfbda38df5a60eec6e85835e4859"}'}

In [3]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

kaggle.json


In [4]:
!kaggle competitions download -c nyc-taxi-trip-duration


  0% 0.00/2.49M [00:00<?, ?B/s]
100% 2.49M/2.49M [00:00<00:00, 83.0MB/s]
 44% 9.00M/20.3M [00:00<00:00, 20.7MB/s]
100% 20.3M/20.3M [00:00<00:00, 37.7MB/s]
 94% 59.0M/62.9M [00:01<00:00, 21.4MB/s]
100% 62.9M/62.9M [00:01<00:00, 59.0MB/s]


In [5]:
!ls

kaggle.json  sample_data  sample_submission.zip  test.zip  train.zip


In [6]:
train_data = pd.read_csv("train.zip")
train_data.head()

,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435


In [7]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   id                  1458644 non-null  object 
 1   vendor_id           1458644 non-null  int64  
 2   pickup_datetime     1458644 non-null  object 
 3   dropoff_datetime    1458644 non-null  object 
 4   passenger_count     1458644 non-null  int64  
 5   pickup_longitude    1458644 non-null  float64
 6   pickup_latitude     1458644 non-null  float64
 7   dropoff_longitude   1458644 non-null  float64
 8   dropoff_latitude    1458644 non-null  float64
 9   store_and_fwd_flag  1458644 non-null  object 
 10  trip_duration       1458644 non-null  int64  
dtypes: float64(4), int64(3), object(4)
memory usage: 122.4+ MB


In [8]:
train_data.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [ ]:
def pandas_profile(df):
    print(train_data.describe())
    print(train_data.info())
    from pandas_profiling import ProfileReport
    prof = ProfileReport(df)
    prof.to_file(output_file='report.html')

In [ ]:
train_data['pickup_datetime'] = pd.to_datetime(train_data['pickup_datetime'])
train_data['dropoff_datetime'] = pd.to_datetime(train_data['dropoff_datetime'])

In [ ]:
def datetime_to_float(d):
    return d.timestamp()
temp = train_data['pickup_datetime'].apply(datetime_to_float)

In [ ]:
train_data_cpy = train_data.drop(['id'], axis=1)

In [ ]:
def datetime_to_float(d):
    return d.timestamp()

def normalize_numeric_data(data):
    y = data['trip_duration']
    y_train = np.asarray(y).astype('float32')
    data = data.drop(['trip_duration'], axis=1)
    date_time = data.select_dtypes(include=['datetime64'])
    for column in date_time:
        date_time[column] = date_time[column].apply(datetime_to_float)
    numeric_data = data.select_dtypes(include=[np.number])
    from sklearn import preprocessing
    x = numeric_data.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    df = pd.DataFrame(x_scaled)
    df_col = pd.concat([date_time,df], axis=1)
    #np_reader = tf.contrib.timeseries.NumpyReader(data={tf.contrib.timeseries.TrainEvalFeatures.TIMES: data['timestamp'].values, tf.contrib.timeseries.TrainEvalFeatures.VALUES : data['value'].values})
    X = df_col.to_numpy()
    #X = np.asarray(X).astype(np.float32)
    return X, y_train

In [ ]:
X, y = normalize_numeric_data(train_data)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)

In [ ]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential() 
    model.add(layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],))) 
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1))
    model.compile(optimizer='rmsprop', loss='mse', metrics=['acc']) 
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test),epochs=20, batch_size=1, verbose=1)
    return history